In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
#tf.enable_eager_execution()
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

In [ ]:
nfl = pd.read_csv('data/nfl_scores.csv')

In [ ]:
nfl['schedule_date'] = pd.to_datetime(nfl['schedule_date'])

In [ ]:
nfl.index = nfl['schedule_date']

In [ ]:
nfl = nfl.drop(columns='schedule_date')

In [ ]:
nfl_1 = nfl['2014-09-04':'2019-11-24']

In [ ]:
#normalize weather temp
nfl_1['weather_temperature'].max()
nfl_1['weather_temperature'] = pd.cut(nfl_1['weather_temperature'],
                                     bins = [-10,-5,0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,np.inf],
                                     labels = [-7.5,-2.5,2.5,7.5,12.5,17.5,22.5,27.5,32.5,37.5,42.5,
                                               47.5,52.5,57.5,62.5,67.5,72.5,77.5,82.5,87.5,92.5,97.5,102.5])

In [ ]:
nfl_1['home_win'] = nfl_1['score_home'] > nfl_1['score_away']
    

In [ ]:
nfl_1.home_win = nfl_1.home_win.astype(int)

In [ ]:
undecided = nfl_1

In [ ]:
undecided_2 = undecided

In [ ]:
undecided_2 = undecided_2.replace(['Arizona Cardinals', 
                                  'Atlanta Falcons',
                                  'Baltimore Ravens',
                                   'Buffalo Bills',
                                   'Carolina Panthers',
                                   'Chicago Bears',
                                   'Cincinnati Bengals',
                                   'Cleveland Browns',
                                   'Dallas Cowboys',
                                   'Denver Broncos',
                                   'Detroit Lions',
                                   'Green Bay Packers',
                                   'Houston Texans',
                                   'Indianapolis Colts',
                                   'Jacksonville Jaguars',
                                   'Kansas City Chiefs',
                                   'Los Angeles Chargers',
                                   'Los Angeles Rams',
                                   'Miami Dolphins',
                                   'Minnesota Vikings',
                                   'New Orleans Saints',
                                   'New York Giants',
                                   'New York Jets',
                                   'Oakland Raiders',
                                   'Philadelphia Eagles',
                                   'Pittsburgh Steelers',
                                   'San Francisco 49ers',
                                   'Seattle Seahawks',
                                   'Tampa Bay Buccaneers',
                                   'Tennessee Titans',
                                   'Washington Redskins',
                                  'St. Louis Rams',
                                  'San Diego Chargers',
                                  'New England Patriots',
                                  'Division',
                                  'Conference',
                                  'SuperBowl',
                                   'Superbowl',
                                  'WildCard'],
                                  [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,
                                   18,19,20,21,22,23,24,25,26,27,28,29,30,31,
                                   32,33,34,35,36,37,37, 38])

In [ ]:
#fav home
undecided_2 = undecided_2.replace(['ARI', 
                                  'ATL',
                                  'BAL',
                                   'BUF',
                                   'CAR',
                                   'CHI',
                                   'CIN',
                                   'CLE',
                                   'DAL',
                                   'DEN',
                                   'DET',
                                   'GB',
                                   'HOU',
                                   'IND',
                                   'JAX',
                                   'KC',
                                   'LAC',
                                   'LAR',
                                   'MIA',
                                   'MIN',
                                   'NO',
                                   'NYG',
                                   'NYJ',
                                   'OAK',
                                   'PHI',
                                   'PIT',
                                   'SF',
                                   'SEA',
                                   'TB',
                                   'TEN',
                                   'WAS',
                                  'STR',
                                  'SDC',
                                  'NE',
                                  ],
                                  [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,
                                   18,19,20,21,22,23,24,25,26,27,28,29,30,31,
                                   32,33,34])

In [ ]:
#fav home
undecided_2['home_fav'] = undecided_2['team_home'] == undecided_2['team_favorite_id']
undecided_2.home_fav = undecided_2.home_fav.astype(int)

In [ ]:
features = ['schedule_week', 'weather_temperature', 'weather_wind_mph',
           'team_home', 'team_away', 'spread_favorite', 'over_under_line','home_fav'
            ]

In [ ]:
x = undecided_2.loc[:, features]
x = x.replace('Wildcard', 18, regex=True)

In [ ]:
y = undecided_2.home_win

In [ ]:
#scikit DNN

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
model = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=8*[94], random_state=10)

In [ ]:
#train data one day behind

In [ ]:
train_data = x['2014-09-04':'2019-11-23']

In [ ]:
train_y = undecided_2.home_win

In [ ]:
train_y_data = train_y['2014-09-04':'2019-11-23']

In [ ]:
#day of data
test_data = x['2019-11-10':'2019-11-10']
test_y_data = train_y['2019-11-10':'2019-11-10']

In [ ]:
model.fit(train_data,train_y_data)

In [ ]:
pred = model.predict(x)

In [ ]:
pred = pd.DataFrame(pred)

In [ ]:
nfl_1.tail(12)

In [ ]:
#accuarcy predictor day of
pred_final = pred.tail(12)
y = nfl_1['home_win']
y_final = y.tail(12)
y_final = pd.DataFrame(y_final)
accuracy_score(pred_final, y_final)

In [ ]:
#TTS 3.0 final predictions
pred_final

In [ ]:
stop
print('{:>8s} | {:8s}'.format('depth', 'accuracy'))
print(20 * '-')
for depth in range(90,110):
    model = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=8*[depth], random_state=10)
    model.fit(train_data,train_y_data)
    acc = accuracy_score(test_y_data, model.predict(test_data))
    print('{:>8d} | {:8.2f}'.format(depth, acc))

In [ ]:
accuracy_score(pred,y)

In [ ]:
#decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model = DecisionTreeClassifier(max_depth=28,random_state=50)

In [ ]:
model.fit(train_data, train_y_data)

In [ ]:
pred = model.predict(x)

In [ ]:
pred = pd.DataFrame(pred)

In [ ]:
#decision tree predictions
pred.tail(12)

In [ ]:
final_pred = pred.tail(11)
y = nfl_1['home_win']
y_final = y.tail(11)
y_final = pd.DataFrame(y_final)

In [ ]:
accuracy_score(final_pred,y_final)

In [ ]:
stop
print('{:>8s} | {:8s}'.format('depth', 'accuracy'))
print(20 * '-')
for depth in range(10,100,10):
    model = DecisionTreeClassifier(max_depth=depth, random_state = 50)
    model.fit(train_data,train_y_data)
    acc = accuracy_score(test_y_data, model.predict(test_data))
    print('{:>8d} | {:8.2f}'.format(depth, acc))

In [ ]:
#scalar data

In [ ]:
#from sklearn import preprocessing

In [ ]:
#scaler = preprocessing.MinMaxScaler()

In [ ]:
#train_data = scaler.fit_transform(train_data)

In [ ]:
#train_data = pd.DataFrame(train_data)

In [ ]:
#train_data

In [ ]:
#pred

In [ ]:
#pred[0].value_counts()

In [ ]:
#SGDClassifier
from sklearn.linear_model import SGDClassifier

In [ ]:
model = SGDClassifier(loss="hinge", penalty="l2", max_iter=100, random_state=42)

In [ ]:
model.fit(train_data, train_y_data)

In [ ]:
pred = model.predict(x)

In [ ]:
pred = pd.DataFrame(pred)

In [ ]:
pred.tail(12)

In [ ]:
final_pred = pred.tail(13)
y = nfl_1['home_win']
y_final = y.tail(13)
y_final = pd.DataFrame(y_final)

In [ ]:
stop
print('{:>8s} | {:8s}'.format('depth', 'accuracy'))
print(20 * '-')
for depth in range(10,200,10):
    model = SGDClassifier(loss="hinge", penalty="l2", max_iter=depth)
    model.fit(train_data,train_y_data)
    acc = accuracy_score(test_y_data, model.predict(test_data))
    print('{:>8d} | {:8.2f}'.format(depth, acc))

In [ ]:
accuracy_score(final_pred, y_final)

In [ ]:
#gradient boosting classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
        max_depth=1, random_state=0, loss='deviance')

In [ ]:
model.fit(train_data, train_y_data)

In [ ]:
pred = model.predict(x)
pred = pd.DataFrame(pred)

In [ ]:
pred.tail(12)

In [ ]:
accuracy_score(pred,y)

In [ ]:
#random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [ ]:
model = RandomForestClassifier(n_estimators=100, max_depth=1,
                               random_state=0)

In [ ]:
model.fit(train_data, train_y_data)

In [ ]:
pred = model.predict(x)
pred = pd.DataFrame(pred)

In [ ]:
accuracy_score(pred,y)

In [ ]:
stop
print('{:>8s} | {:8s}'.format('depth', 'accuracy'))
print(20 * '-')
for depth in range(1,15):
    model = RandomForestClassifier(n_estimators=100, max_depth=depth,
                               random_state=0)
    model.fit(train_data,train_y_data)
    acc = accuracy_score(test_y_data, model.predict(test_data))
    print('{:>8d} | {:8.2f}'.format(depth, acc))

In [ ]:
pred.tail(12)

In [ ]:
nfl_1['home_win'].value_counts()

In [ ]:
x.tail(12)

In [ ]:
undecided_2.tail(12)